In [1]:
### Run this notebook using cell -> run all
### Demo of one solution to 'blocking' until a widget value has changed

In [2]:
shell = get_ipython()
shell

In [3]:
kernel = shell.kernel
kernel

In [4]:
captured_events = []

def execute_request(stream, ident, parent):
    "Overwrite function to store the stream / ident /parent instead of calling kernel.execute_request"
    
    captured_events.append((stream, ident, parent))
    
def comm_msg(stream, ident, parent):
    "Overwrite function to add a logging (print) msg when comm_msg events come through"
    print("in comm_msg")
    print(stream)
    print(ident)
    print(parent)
    return kernel.comm_manager.comm_msg(stream, ident, parent)
    
def start_capturing():
    "Overwrite the kernel shell handlers"
    kernel.shell_handlers['execute_request'] = execute_request
    kernel.shell_handlers['comm_msg'] = comm_msg
    
def stop_capturing():
    "rever the kernel shell handler functions to their defaults"
    kernel.shell_handlers['execute_request'] = kernel.execute_request
    kernel.shell_handlers['comm_msg'] = kernel.comm_manager.comm_msg

In [8]:
# Display a widget and 'block' (don't run execute_request messages) until the widget has changed value
import ipywidgets as widgets
from IPython.display import display
import time

w = widgets.Dropdown(options=['', 'foo', 'bar'])

start_capturing() # override kernel.shell_handler functions
display(w) 
shell.execution_count += 1

while True:
    # While this loop is running, all further 'execute_request' messages will get captured
    if w.value:
        print("widget value changed: breaking from loop")
        break # user changed the value
    
    kernel.do_one_iteration() # same thing an eventloop like %gui asyncio would do
    time.sleep(.01)

stop_capturing()
import sys
sys.stdout.flush()
### Once the widget value has changed, 'replay' the captured execute_request messages
### Unfortunately the output shows up in this cell, not in the cells where the
### original input code is at...
for stream, ident, parent in captured_events:
    kernel.set_parent(ident, parent)
    kernel.execute_request(stream, ident, parent)

Dropdown(options=('', 'foo', 'bar'), value='')

in comm_msg
[b'1aeed0a8e4844cc88be1b0d2c2eefcef']
{'header': {'msg_id': '3b5a6598ca4e4ce989cd3cb1cf6c4bb4', 'username': 'username', 'session': '1aeed0a8e4844cc88be1b0d2c2eefcef', 'msg_type': 'comm_msg', 'version': '5.2', 'date': datetime.datetime(2018, 8, 25, 18, 45, 18, 508894, tzinfo=tzutc())}, 'msg_id': '3b5a6598ca4e4ce989cd3cb1cf6c4bb4', 'msg_type': 'comm_msg', 'parent_header': {}, 'metadata': {}, 'content': {'comm_id': 'c78081bbe7914bc4a5b94b1b64565ecb', 'data': {'method': 'update', 'state': {'index': 1}, 'buffer_paths': []}}, 'buffers': []}


In [6]:
# this should get captured in a cell -> run all (last cell is still running/widget value hasn't changed yet)
print(w.value)

widget value changed: breaking from loop
foo


In [7]:
# this should get captured in a cell -> run all 
print("hello world")

hello world
